In [1]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torchvision.datasets as datasets 
import torchvision.transforms as transforms 
import matplotlib.pyplot as plt 

In [2]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder,self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2D(3,16,3,stride=1,padding=1),
            nn.ReLu(),
            nn.MaxPool2d(2,stride=2),
            nn.lConv2D(16,8,3,stride=1,padding=1),
            nn.ReLu(),
            nn.MaxPool2d(2,stride=2)


        )

        self.decoder=nn.sequential(
            nn.ConvTranspose2d(8,16,3,stride=2,padding=1,ooutput_padding=1),
            nn.ReLu(),
            nn.ConvTranspose2d(16,3,3,stride=2,padding=1,output_padding=1),
            nn.Sigmoid()

        )
        def forward(self,x):
            x=self.encoder(x)
            x=self.decoder(x)
            return x 

data preparation : transformer and loader 

In [3]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])
train_dataset = datasets.Flowers102(root='flowers', split='train', transform=transform, download=True)
test_dataset = datasets.Flowers102(root='flowers', split='test', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128)

100%|██████████| 344862509/344862509 [03:19<00:00, 1724623.35it/s]


Extracting flowers/flowers-102/102flowers.tgz to flowers/flowers-102


100%|██████████| 502/502 [00:00<00:00, 1619646.62it/s]


100%|██████████| 14989/14989 [00:00<00:00, 28865207.83it/s]


In [4]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device is available:{device}")

Device is available:cpu
